In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install -q pytorch-metric-learning[with-hooks]
!pip install umap-learn

In [ ]:
# sample_ids = pd.read_csv('/kaggle/input/sample-ids/Samples 1000.csv')

In [ ]:
# sample_ids = sample_ids['landmark_id']

In [ ]:
# sample_ids.columns = ['landmark_id']

In [ ]:
# data = pd.read_csv('/kaggle/input/landmark-recognition-2020/train.csv')

In [ ]:
# sample_data = data.loc[data['landmark_id'].isin(sample_ids['landmark_id'].values)]

In [ ]:
# sample_data

In [ ]:
# sample_data = sample_data.reset_index()

In [ ]:
# sample_data.loc[sample_data['landmark_id'].value_counts() == 2]

In [ ]:
# train_path = '/kaggle/input/landmark-recognition-2020/train/'
# file_paths = []
# for root, dirs, files in os.walk(train_path):
#     for file in files:
#         file_paths.append(os.path.join(root,file))

In [ ]:
# len(file_paths)

In [ ]:
# file_paths[0]

In [ ]:
# file_name = [idx.split('/')[-1].split('.')[0] for idx in file_paths]

In [ ]:
# file_df = pd.DataFrame(file_name, columns=['id'])

In [ ]:
# file_df['file_path'] = file_paths

In [ ]:
# file_df

In [ ]:
# sample_files = file_df.loc[file_df['id'].isin(sample_data['id'])]

In [ ]:
# sample_df = pd.merge(sample_data, sample_files)

In [ ]:
# sample_df.iloc[0]['id']

In [ ]:
# sample_df.iloc[0]['file_path']

In [ ]:
# sample_df

In [ ]:
# sample_df.to_csv('/kaggle/working/sample_df')

In [ ]:
%matplotlib inline
from pytorch_metric_learning import losses, miners, samplers, trainers, testers
from pytorch_metric_learning.utils import common_functions
import pytorch_metric_learning.utils.logging_presets as logging_presets
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator
import numpy as np
import torchvision
from torchvision import datasets, transforms
import torch
import torch.nn as nn
from PIL import Image
import logging
import matplotlib.pyplot as plt
import umap
from cycler import cycler
import record_keeper
import pytorch_metric_learning
import cv2
from PIL import Image
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
logging.getLogger().setLevel(logging.INFO)
logging.info("VERSION %s"%pytorch_metric_learning.__version__)

In [ ]:
sample_df = pd.read_csv('/kaggle/input/sample-ids/sample_df')

In [ ]:
class MLP(nn.Module):
    # layer_sizes[0] is the dimension of the input
    # layer_sizes[-1] is the dimension of the output
    def __init__(self, layer_sizes, final_relu=False):
        super().__init__()
        layer_list = []
        layer_sizes = [int(x) for x in layer_sizes]
        num_layers = len(layer_sizes) - 1
        final_relu_layer = num_layers if final_relu else num_layers - 1
        for i in range(len(layer_sizes) - 1):
            input_size = layer_sizes[i]
            curr_size = layer_sizes[i + 1]
            if i < final_relu_layer:
                layer_list.append(nn.ReLU(inplace=False))
            layer_list.append(nn.Linear(input_size, curr_size))
        self.net = nn.Sequential(*layer_list)
        self.last_linear = self.net[-1]

    def forward(self, x):
        return self.net(x)

In [ ]:
pip install timm

In [ ]:
import timm

In [ ]:
#!pip install git+https://github.com/qubvel/classification_models.git

In [ ]:
# for keras
#from classification_models.keras import Classifiers

# for tensorflow keras
#from classification_models.tfkeras import Classifiers

#Classifiers.models_names()

In [ ]:
#SeResNeXT101, preprocess_input = Classifiers.get('seresnext101')


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set trunk model and replace the softmax layer with an identity function
trunk = timm.create_model('seresnet101', pretrained=True)
#trunk = SeResNeXT101(include_top = False, input_shape=(224, 224, 3), weights='imagenet')
#trunk = torchvision.models.resnet50(pretrained=True)
trunk_output_size = trunk.fc.in_features
trunk.fc = common_functions.Identity()
trunk = torch.nn.DataParallel(trunk.to(device))

# Set embedder model. This takes in the output of the trunk and outputs 64 dimensional embeddings
embedder = torch.nn.DataParallel(MLP([trunk_output_size, 256]).to(device))

# Set optimizers
trunk_optimizer = torch.optim.Adam(trunk.parameters(), lr=0.0001, weight_decay=0.0001)
embedder_optimizer = torch.optim.Adam(embedder.parameters(), lr=0.0001, weight_decay=0.0001)

# Set the image transforms
transform = transforms.Compose([transforms.Resize((224, 224)),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])


In [ ]:
# ct = 0
# for param in trunk.parameters():
#     if ct <=150:
#         param.requires_grad = False  
#     ct+=1

In [ ]:
trunk_output_size

In [ ]:
# for param in trunk.parameters():
#     print(param.requires_grad)

In [ ]:
class create_dataset(torch.utils.data.Dataset):
    def __init__(self, data_df, transform):
        self.data_df = data_df
        self.transform = transform 
    
    def __len__(self):
        return self.data_df.shape[0]
    
    def __getitem__(self, idx):
        try:
            img = Image.open(self.data_df.iloc[idx]['file_path'])
            target = self.data_df.iloc[idx]['landmark_id']

            if self.transform is not None:
                img = self.transform(img)

            return img, target
        except:
            pass

In [ ]:
c = create_dataset(sample_df, transform)

In [ ]:
img, target = c.__getitem__(10)

In [ ]:
plt.imshow(np.moveaxis(np.asarray(img), 0, -1))

In [ ]:
img

In [ ]:
StratifiedShuffleSplit()

In [ ]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)

In [ ]:
sample_df['train'] = 1

for train_index, val_index in sss.split(sample_df, sample_df['landmark_id']):
    sample_df['train'].iloc[val_index] = 0

In [ ]:
val_df = sample_df.loc[sample_df['train'] == 0]
train_df = sample_df.loc[sample_df['train'] == 1]

In [ ]:
train_dataset = create_dataset(train_df, transform)
val_dataset = create_dataset(val_df, transform)

In [ ]:
# train_dataset.data_df['landmark_id'].nunique()

In [ ]:
embedder.parameters

In [ ]:
loss = losses.TripletMarginLoss(margin=2.0)

# Set the mining function
miner = miners.BatchEasyHardMiner()

# Set the dataloader sampler
# sampler = samplers.MPerClassSampler(train_dataset.targets, m=4, length_before_new_iter=len(train_dataset))

# Set other training parameters
batch_size = 64
num_epochs = 30

# Package the above stuff into dictionaries.
models = {"trunk": trunk, "embedder": embedder}
optimizers = {"trunk_optimizer": trunk_optimizer, "embedder_optimizer": embedder_optimizer}
loss_funcs = {"metric_loss": loss}
mining_funcs = {"tuple_miner": miner}
# mining_funcs = {}

In [ ]:
record_keeper, _, _ = logging_presets.get_record_keeper("example_logs", "example_tensorboard")
hooks = logging_presets.get_hook_container(record_keeper)
dataset_dict = {"val": val_dataset}
model_folder = "example_saved_models"

tester = testers.GlobalEmbeddingSpaceTester(end_of_testing_hook = hooks.end_of_testing_hook, 
                                            dataloader_num_workers = 32,
                                            accuracy_calculator=AccuracyCalculator(k="max_bin_count"))


end_of_epoch_hook = hooks.end_of_epoch_hook(tester, 
                                            dataset_dict, 
                                            model_folder, 
                                            test_interval = 1,
                                            patience = 1)

In [ ]:
trainer = trainers.MetricLossOnly(models,
                                optimizers,
                                batch_size,
                                loss_funcs,
                                mining_funcs,
                                train_dataset,
                                dataloader_num_workers = 32,
                                end_of_iteration_hook = hooks.end_of_iteration_hook,
                                end_of_epoch_hook = end_of_epoch_hook)

In [ ]:
val_df['landmark_id'].nunique()

In [ ]:
trainer.train(num_epochs=num_epochs)